# Getting Started: Install QPyTorch 

In [1]:
!pip install -r requirements.txt

  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.6 MB/s eta 0:00:00
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 47.9 MB/s eta 0:00:0000:0100:01
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 48.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 33.6 MB/s eta 0:00:00
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Lab 1. Quantizer Tutorial

In [2]:
import torch
from qtorch.quant import Quantizer, quantizer
from qtorch import FloatingPoint

/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


### Input Tensor

In [8]:
random_input = torch.rand([3,3])
print(random_input)

tensor([[0.9896, 0.2558, 0.0652],
        [0.2630, 0.6916, 0.1466],
        [0.0170, 0.6570, 0.6628]])


In [12]:
constant_input = torch.tensor([[255., -255.],[1.4, 1.6]])
print(constant_input)

tensor([[ 255.0000, -255.0000],
        [   1.4000,    1.6000]])


### Quantization

In [9]:
# Example: FP4 (E2M1) Quantization 0 ~ 15
bit = FloatingPoint(exp=2, man=1) # 한개는 sign
quant = quantizer(forward_number=bit, forward_rounding="nearest")

In [27]:
quant

<bound method Function.apply of <class 'qtorch.quant.quant_function.quantizer.<locals>.Rounding'>>

In [10]:
random_output = quant(random_input)
print(random_output)

tensor([[1.0000, 0.5000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.0000, 0.5000, 0.5000]])


In [13]:
constant_output = quant(constant_input)
print(constant_output) # -6 ~ 6 -> 양자화 후 -255, 255 범위 내 표현할 수 있는 최대값

tensor([[ 6.0000, -6.0000],
        [ 1.5000,  1.5000]])


# Lab 2. Reduced-Precision Training

In [14]:
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from qtorch.quant import Quantizer, quantizer
from qtorch.optim import OptimLP
from torch.optim import SGD
from qtorch import FloatingPoint
from tqdm import tqdm
import math

In [15]:
# loading data
ds = torchvision.datasets.CIFAR10
path = os.path.join("./data", "CIFAR10")
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set = ds(path, train=True, download=True, transform=transform_train)
test_set = ds(path, train=False, download=True, transform=transform_test)
loaders = {
        'train': torch.utils.data.DataLoader(
            train_set,
            batch_size=128,
            shuffle=True,
            num_workers=4,
            pin_memory=True
        ),
        'test': torch.utils.data.DataLoader(
            test_set,
            batch_size=128,
            num_workers=4,
            pin_memory=True
        )
}

100%|██████████| 170498071/170498071 [00:23<00:00, 7396329.69it/s] 


Extracting ./data/CIFAR10/cifar-10-python.tar.gz to ./data/CIFAR10
Files already downloaded and verified


In [18]:
# define two floating point formats
bit_8 = FloatingPoint(exp=5, man=2)
bit_16 = FloatingPoint(exp=6, man=9)

# define quantization functions
weight_quant = quantizer(forward_number=bit_8,
                        forward_rounding="nearest")
grad_quant = quantizer(forward_number=bit_8,
                        forward_rounding="nearest")
momentum_quant = quantizer(forward_number=bit_16,
                        forward_rounding="stochastic")
acc_quant = quantizer(forward_number=bit_16,
                        forward_rounding="stochastic")

# define a lambda function so that the Quantizer module can be duplicated easily
act_error_quant = lambda : Quantizer(forward_number=bit_8, backward_number=bit_8,
                        forward_rounding="nearest", backward_rounding="nearest")

In [19]:
# Define model
# 동일한 Qunt를 사용?
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, quant, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.downsample = downsample
        self.stride = stride
        self.quant = quant()

    def forward(self, x):
        residual = x

        ####### FIXME #######
        out = self.bn1(x)
        out = self.relu(out)
        out = self.quant(out)
        out = self.conv1(out)
        out = self.quant(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.quant(out)
        out = self.conv2(out)
        out = self.quant(out)
        #####################

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out
    
class PreResNet(nn.Module):

    def __init__(self,quant, num_classes=10, depth=20):

        super(PreResNet, self).__init__()
        assert (depth - 2) % 6 == 0, 'depth should be 6n+2'
        n = (depth - 2) // 6

        block = BasicBlock

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1,
                               bias=False)
        self.layer1 = self._make_layer(block, 16, n, quant)
        self.layer2 = self._make_layer(block, 32, n, quant, stride=2)
        self.layer3 = self._make_layer(block, 64, n, quant, stride=2)
        self.bn = nn.BatchNorm2d(64 * block.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64 * block.expansion, num_classes)
        self.quant = quant()
        IBM_half = FloatingPoint(exp=6, man=9)
        self.quant_half = Quantizer(IBM_half, IBM_half, "nearest", "nearest")
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, quant, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
            )

        layers = list()
        layers.append(block(self.inplanes, planes, quant , stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, quant))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.quant_half(x)
        x = self.conv1(x)
        x = self.quant(x) # Quantization

        x = self.layer1(x)  # 32x32
        x = self.layer2(x)  # 16x16
        x = self.layer3(x)  # 8x8
        x = self.bn(x)
        x = self.relu(x)
        x = self.quant(x) # Quantization

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.quant_half(x) # Quantization

        return x

In [20]:
model = PreResNet(act_error_quant)

In [21]:
device = 'cuda' # change device to 'cuda' if you want to run this example on cuda
model = model.to(device=device)

In [22]:
optimizer = SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4)
optimizer = OptimLP(optimizer,
                    weight_quant=weight_quant,
                    grad_quant=grad_quant,
                    momentum_quant=momentum_quant,
                    acc_quant=acc_quant,
                    grad_scaling=1/1000 # do loss scaling
)

In [23]:
def run_epoch(loader, model, criterion, optimizer=None, phase="train"):
    assert phase in ["train", "eval"], "invalid running phase"
    loss_sum = 0.0
    correct = 0.0

    if phase=="train": model.train()
    elif phase=="eval": model.eval()

    ttl = 0
    with torch.autograd.set_grad_enabled(phase=="train"):
        for i, (input, target) in tqdm(enumerate(loader), total=len(loader)):
            input = input.to(device=device)
            target = target.to(device=device)
            output = model(input)
            loss = criterion(output, target)
            loss_sum += loss.cpu().item() * input.size(0)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            ttl += input.size()[0]

            if phase=="train":
                loss = loss * 1000 # do loss scaling
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

    correct = correct.cpu().item()
    return {
        'loss': loss_sum / float(ttl),
        'accuracy': correct / float(ttl) * 100.0,
    }

### Accuracy Before Training

In [24]:
test_res = run_epoch(loaders['test'], model, F.cross_entropy,
                        optimizer=optimizer, phase="eval")
print(test_res)

100%|██████████| 79/79 [00:00<00:00, 83.95it/s] 

{'loss': 3.308834511566162, 'accuracy': 10.36}


### 3-Epoch Training

In [25]:
for epoch in range(3):
    train_res = run_epoch(loaders['train'], model, F.cross_entropy,
                                optimizer=optimizer, phase="train")
    print(f'=====> Epoch {epoch}')
    print(f'Train loss    : {train_res["loss"]}')

100%|██████████| 391/391 [00:10<00:00, 37.46it/s]

=====> Epoch 0
Train loss    : 1.6348645886611939



100%|██████████| 391/391 [00:10<00:00, 38.11it/s]

=====> Epoch 1
Train loss    : 1.141644220275879



100%|██████████| 391/391 [00:10<00:00, 38.16it/s]

=====> Epoch 2
Train loss    : 0.9543998905563355


### Accuracy After Training

In [26]:
test_res = run_epoch(loaders['test'], model, F.cross_entropy,
                        optimizer=optimizer, phase="eval")
print(test_res)

100%|██████████| 79/79 [00:00<00:00, 122.47it/s]

{'loss': 1.0747511833190917, 'accuracy': 64.05999999999999}


# Lab 3.1. Trans-Precision Inference on CNN Models

In [28]:
# Define model
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, quant, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.downsample = downsample
        self.stride = stride
        self.quant = quant()

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.quant(out)
        out = self.conv1(out)
        out = self.quant(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.quant(out)
        out = self.conv2(out)
        out = self.quant(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out
    
class PreResNet(nn.Module):

    def __init__(self,quant, num_classes=10, depth=20):

        super(PreResNet, self).__init__()
        assert (depth - 2) % 6 == 0, 'depth should be 6n+2'
        n = (depth - 2) // 6

        block = BasicBlock

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1,
                               bias=False)
        self.layer1 = self._make_layer(block, 16, n, quant)
        self.layer2 = self._make_layer(block, 32, n, quant, stride=2)
        self.layer3 = self._make_layer(block, 64, n, quant, stride=2)
        self.bn = nn.BatchNorm2d(64 * block.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64 * block.expansion, num_classes)
        self.quant = quant()
        IBM_half = FloatingPoint(exp=6, man=9)
        self.quant_half = Quantizer(IBM_half, IBM_half, "nearest", "nearest")
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, quant, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
            )

        layers = list()
        layers.append(block(self.inplanes, planes, quant , stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, quant))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.quant_half(x)
        x = self.conv1(x)
        x = self.quant(x)

        x = self.layer1(x)  # 32x32
        x = self.layer2(x)  # 16x16
        x = self.layer3(x)  # 8x8
        x = self.bn(x)
        x = self.relu(x)
        x = self.quant(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.quant_half(x)

        return x

In [29]:
# define Three floating point formats
bit_8 = FloatingPoint(exp=5, man=2)
bit_16 = FloatingPoint(exp=6, man=9)
bit_32 = FloatingPoint(exp=8, man=23)

# define a lambda function so that the Quantizer module can be duplicated easily
act_error_quant = lambda : Quantizer(forward_number=bit_8, backward_number=bit_8,
                        forward_rounding="nearest", backward_rounding="nearest")

In [30]:
model = PreResNet(act_error_quant)

In [31]:
checkpoint = torch.load('./PreResNet_fp32.pth')
model.load_state_dict(checkpoint['model'])

/tmp/ipykernel_1106140/477816878.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./PreResNet_fp32.pth')


<All keys matched successfully>

In [32]:
device = 'cuda' # change device to 'cuda' if you want to run this example on cuda
model = model.to(device=device)

In [33]:
def run_test(loader, model, criterion, optimizer=None):
    loss_sum = 0.0
    correct = 0.0

    model.eval()

    ttl = 0
    with torch.no_grad():
        for i, (input, target) in tqdm(enumerate(loader), total=len(loader)):
            input = input.to(device=device)
            target = target.to(device=device)
            output = model(input)
            loss = criterion(output, target)
            loss_sum += loss.cpu().item() * input.size(0)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            ttl += input.size()[0]

    correct = correct.cpu().item()
    return {
        'loss': loss_sum / float(ttl),
        'accuracy': correct / float(ttl) * 100.0,
    }

In [34]:
test_res = run_test(loaders['test'], model, F.cross_entropy, optimizer=optimizer)
print(test_res)

100%|██████████| 79/79 [00:00<00:00, 123.08it/s]

{'loss': 0.2672718163609505, 'accuracy': 92.10000000000001}


# Lab 3.2. Trans-Precision Inference on LLMs

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [36]:
model_id = "Qwen/Qwen2-0.5B-Instruct"
device = "cuda" # the device to load the model onto

LLM = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    attn_implementation="eager",
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
prompt = "Give me a short introduction of Samsung Electronics."

In [38]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = LLM.generate(
    model_inputs.input_ids,
    num_beams=1,
    do_sample=False,
    max_new_tokens=128
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Samsung Electronics is one of the world's largest and most successful electronics companies, with a presence in over 190 countries. The company was founded in 1938 by Lee Dong-hwan and has since grown to become one of the world's leading producers of consumer electronics, including smartphones, TVs, computers, and other devices.

Samsung Electronics operates several major divisions, including the Mobile Phone Division, which produces smartphones, tablets, and other mobile devices; the TV Division, which produces TVs, LCD monitors, and other home entertainment products; the Consumer Electronics Division, which produces a wide range of consumer electronics products, including televisions


## Define Quantized Modules

In [49]:
from typing import List, Optional, Tuple, Union
from transformers.cache_utils import Cache, DynamicCache

class MatMul(nn.Module):
    def forward(self, A, B):
        return A @ B

class QuantMatMul(nn.Module):
    def __init__(self, qbit):
        super().__init__()
        self.quantizer = Quantizer(forward_number=qbit, backward_number=qbit,
                     forward_rounding="nearest", backward_rounding="nearest")
    def forward(self, A, B):
        qA = self.quantizer.quantize(A.data)
        qB = self.quantizer.quantize(B.data)
        return qA @ qB

from transformers.models.qwen2.modeling_qwen2 import rotate_half, apply_rotary_pos_emb, repeat_kv

def attn_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_value: Optional[Cache] = None,
    output_attentions: bool = False,
    use_cache: bool = False,
    **kwargs,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    if "padding_mask" in kwargs:
        warnings.warn(
            "Passing `padding_mask` is deprecated and will be removed in v4.37. Please make sure use `attention_mask` instead.`"
        )
    bsz, q_len, _ = hidden_states.size()

    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None:
        if self.layer_idx is None:
            raise ValueError(
                f"The cache structure has changed since version v4.36. If you are using {self.__class__.__name__} "
                "for auto-regressive decoding with k/v caching, please make sure to initialize the attention class "
                "with a layer index."
            )
        kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
    cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
    query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

    if past_key_value is not None:
        cache_kwargs = {"sin": sin, "cos": cos}  # Specific to RoPE models
        key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)

    # repeat k/v heads if n_kv_heads < n_heads
    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    attn_weights = self.matmul1(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

    if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):
        raise ValueError(
            f"Attention weights should be of size {(bsz, self.num_heads, q_len, kv_seq_len)}, but is"
            f" {attn_weights.size()}"
        )

    if attention_mask is not None:
        if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
            raise ValueError(
                f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
            )

        attn_weights = attn_weights + attention_mask

    # upcast attention to fp32
    attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
    attn_weights = nn.functional.dropout(attn_weights, p=self.attention_dropout, training=self.training)
    attn_output = self.matmul2(attn_weights, value_states)

    if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
        raise ValueError(
            f"`attn_output` should be of size {(bsz, self.num_heads, q_len, self.head_dim)}, but is"
            f" {attn_output.size()}"
        )

    attn_output = attn_output.transpose(1, 2).contiguous()
    attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)

    attn_output = self.o_proj(attn_output)

    if not output_attentions:
        attn_weights = None

    return attn_output, attn_weights, past_key_value

In [48]:
class QuantLinear(nn.Linear):
    def __init__(self,in_features,out_features,bias,qbit):
        super().__init__(in_features,out_features,bias)
        self.quantizer_x = Quantizer(forward_number=qbit, backward_number=qbit,
                     forward_rounding="nearest", backward_rounding="nearest")
        self.quantizer_w = Quantizer(forward_number=qbit, backward_number=qbit,
                     forward_rounding="nearest", backward_rounding="nearest")
    def forward(self,x):
        w = self.weight.clone()
        qx = self.quantizer_x.quantize(x.data)
        qw = self.quantizer_w.quantize(w.data)
        return F.linear(qx,qw,self.bias)

In [40]:
LLM

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_features=8

In [45]:
LLM.model.layers[0].self_attn

Qwen2Attention(
  (q_proj): Linear(in_features=896, out_features=896, bias=True)
  (k_proj): Linear(in_features=896, out_features=128, bias=True)
  (v_proj): Linear(in_features=896, out_features=128, bias=True)
  (o_proj): Linear(in_features=896, out_features=896, bias=False)
  (rotary_emb): Qwen2RotaryEmbedding()
)

In [46]:
from types import MethodType

def get_QLLM(LLM, qbit):
    from transformers.models.qwen2.modeling_qwen2 import Qwen2Attention
    for name, module in LLM.named_modules():
        if isinstance(module, Qwen2Attention):
            setattr(module, "matmul1", QuantMatMul(qbit))
            setattr(module, "matmul2", QuantMatMul(qbit))
            module.forward = MethodType(attn_forward, module)
            
    wrapped_modules={}
    module_dict={}
    it=[(name,m) for name,m in LLM.named_modules()]
    for name,m in it:
        module_dict[name]=m
        idx=name.rfind('.')
        if idx==-1:
            idx=0
        father_name=name[:idx]
        if father_name in module_dict:
            father_module=module_dict[father_name]
        else:
            raise RuntimeError(f"father module {father_name} not found")
        if isinstance(m,nn.Linear) and 'head' not in name:
            idx = idx+1 if idx != 0 else idx
            new_m = QuantLinear(m.in_features,m.out_features,m.bias is not None,qbit=qbit)
            new_m.weight.data=m.weight.data
            new_m.bias=m.bias
            replace_m=new_m
            wrapped_modules[name] = new_m
            setattr(father_module,name[idx:],replace_m)
    LLM.eval()
    return wrapped_modules

In [50]:
qbit = FloatingPoint(exp=5, man=2)
wrapped_modules = get_QLLM(LLM, qbit)

In [55]:
LLM

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): QuantLinear(
            in_features=896, out_features=896, bias=True
            (quantizer_x): Quantizer()
            (quantizer_w): Quantizer()
          )
          (k_proj): QuantLinear(
            in_features=896, out_features=128, bias=True
            (quantizer_x): Quantizer()
            (quantizer_w): Quantizer()
          )
          (v_proj): QuantLinear(
            in_features=896, out_features=128, bias=True
            (quantizer_x): Quantizer()
            (quantizer_w): Quantizer()
          )
          (o_proj): QuantLinear(
            in_features=896, out_features=896, bias=False
            (quantizer_x): Quantizer()
            (quantizer_w): Quantizer()
          )
          (rotary_emb): Qwen2RotaryEmbedding()
          (matmul1): QuantMatMul(
            (

In [54]:
LLM.model.layers[0].self_attn

Qwen2Attention(
  (q_proj): QuantLinear(
    in_features=896, out_features=896, bias=True
    (quantizer_x): Quantizer()
    (quantizer_w): Quantizer()
  )
  (k_proj): QuantLinear(
    in_features=896, out_features=128, bias=True
    (quantizer_x): Quantizer()
    (quantizer_w): Quantizer()
  )
  (v_proj): QuantLinear(
    in_features=896, out_features=128, bias=True
    (quantizer_x): Quantizer()
    (quantizer_w): Quantizer()
  )
  (o_proj): QuantLinear(
    in_features=896, out_features=896, bias=False
    (quantizer_x): Quantizer()
    (quantizer_w): Quantizer()
  )
  (rotary_emb): Qwen2RotaryEmbedding()
  (matmul1): QuantMatMul(
    (quantizer): Quantizer()
  )
  (matmul2): QuantMatMul(
    (quantizer): Quantizer()
  )
)

In [51]:
prompt = "Give me a short introduction of Samsung Electronics."

In [52]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = LLM.generate(
    model_inputs.input_ids,
    num_beams=1,
    do_sample=False,
    max_new_tokens=128
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/devrok/.conda/envs/qpt/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Samsung Electronics is a South Korean multinational corporation headquartered in Seoul, South Korea. It was founded on September 25, 1968 by Lee Dong-gun and his brother-in-law Lee Dong-hun. The company's main business is the production and sale of electronic devices such as smartphones, tablets, TVs, and other consumer electronics.

Samsung Electronics has been one of the leading companies in the global electronics industry for many years. It has a strong presence in various regions around the world including Asia, Europe, North America, and Africa. 

The company operates through its subsidiaries like Samsung Electronics Co., Ltd., Samsung Display Co., Ltd


In [ ]:
# MXformat ---> 대체 가능한 모듈